In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../data/interim/dataset.csv')
df.head()

,year,GEO,Compensation of employees per hour worked (Euro),Compensation per employee (Euro),education
0,2010,Belgium,34.3,49054.8,48.457143
1,2011,Belgium,35.1,50593.5,47.857143
2,2012,Belgium,36.2,52178.2,48.242857
3,2013,Belgium,37.2,53452.5,49.242857
4,2014,Belgium,37.6,53965.2,50.285714


In [3]:
scaler = StandardScaler()
df['per_hour_worked'] = scaler.fit_transform(df[['Compensation of employees per hour worked (Euro)']])
df['per_employee'] = scaler.fit_transform(df[['Compensation per employee (Euro)']])
df['education'] = scaler.fit_transform(df[['education']])
df[['per_hour_worked', 'per_employee', 'education']].corr()

,per_hour_worked,per_employee,education
per_hour_worked,1.000000,0.990948,0.307323
per_employee,0.990948,1.000000,0.306327
education,0.307323,0.306327,1.000000


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

data = df.dropna()
x = data[['education']]
y = data[['per_hour_worked']]
x_train, x_test, y_train, y_test = train_test_split(x, y)

model = LinearRegression()
model.fit(x_train, y_train)
print(mean_squared_error(y_train, model.predict(x_train)))
print(mean_squared_error(y_test, model.predict(x_test)))
print(model.coef_)

0.8659725434212218
1.0425766113267894
[[0.27039428]]
